In [4]:
import cv2
import numpy as np
import sys
from random import randint

video_path = 'data/itzy.mp4'

'''
if len(sys.argv) != 2:
    print('Input video name is missing')
    exit() 
'''

print('Please Select 3 tracking targets')

cv2.namedWindow("trackingWindow")
cap = cv2.VideoCapture(video_path)
Multitracker = cv2.MultiTracker_create()     # MultiTracker 객체 생성 

# 첫번째 프레임 읽기
ok, frame=cap.read()
if not ok:
    print('Failed to read video')
    exit()    # 커널 종료 
    
bboxes = []
n = 0
while n<2:       # ROI 3번 설정 
    bbox = cv2.selectROI('trackingWindow', frame)   # BoundingBox 설정 
    bboxes.append(bbox)
    n += 1

for bbox in bboxes:
    Multitracker.add(cv2.TrackerMIL_create(), frame, bbox)  # Multitracker.add(알고리즘, 이미지, bbox)
    
'''
OPENCV_OBJECT_TRACKERS = {
"csrt": cv2.TrackerCSRT_create,      # More accurate than KCF but slower. 낮은 프레임(fps25)에서 사용
"kcf": cv2.TrackerKCF_create,        # Fast and accurate
"boosting": cv2.TrackerBoosting_create,
"mil": cv2.TrackerMIL_create,
"tld": cv2.TrackerTLD_create,
"medianflow": cv2.TrackerMedianFlow_create,
"mosse": cv2.TrackerMOSSE_create     # Extremely fast but not as accurate as either KCF or CSRT
}
'''

colors = []
colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )
colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )
#colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )
print(colors)

while cap.isOpened():
    ok, frame=cap.read()        # 동영상을 1프레임씩 읽어온다. -> frame 변수에 넣는다.
                                # 동영상 끝나면 ok = False
    if not ok:
        print ('no image to read')
        break
        
    ## Start timer
    timer = cv2.getTickCount()

    ok, boxes = Multitracker.update(frame)  # frame에서 설정한 boundingBox와 비슷한 물체의 위치를 
                                            # 찾아(Update) 반환한다.
    #print (ok, boxes)
    
    ## Calculate Frames per second (FPS) 
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    
    i = 0
    for newbox in boxes:
        # newbox의 왼쪽, 위쪽, 너비, 높이
        left, top, w, h = [int(v) for v in newbox]
        
        # boxes의 색깔,두께 설정
        cv2.rectangle(frame, pt1=(left, top), pt2=(left + w, top + h), color=colors[i], thickness=3) 
        i += 1
        '''
        pt1 = (int(newbox[0]), int(newbox[1]))
        pt2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, pt1, pt2, (randint(0, 255), randint(0, 255), randint(0, 255)), 2)
        '''
    # Display FPS on frame
    cv2.putText(frame, "FPS : " + str(int(fps)), (140,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
    
    cv2.imshow('trackingWindow', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

Please Select 3 tracking targets
[(227, 64, 135), (206, 80, 117)]
